In [0]:
CREATE OR REFRESH STREAMING TABLE crowd2fund.bronze.transactions
COMMENT 'Transaction Data imported from Crowd2Fund export'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT Date,
       Time,
       `From/To` as From_To,
       `Amount In (GBP)` as Amount_In_GBP,
       `Amount Out (GBP)` as Amount_Out_GBP,
       `Balance (GBP)` as Balance_GBP,
       `Campaign/Business` as Campaign_Business,
      _metadata.file_path AS file_path,
      CURRENT_TIMESTAMP AS ingestion_timestamp 
  FROM STREAM read_files(
      '/Volumes/crowd2fund/landing/exports/transactions',
      format => 'csv',
      delimiter => ',',
      header => true
  );

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW crowd2fund.silver.add_funds (
  CONSTRAINT valid_Amount EXPECT (Amount IS NOT NULL) ON VIOLATION DROP ROW
)
AS
SELECT Date,
       Time,
       Amount_In_GBP as Amount
FROM crowd2fund.bronze.transactions
WHERE From_To = 'Add Funds'


In [0]:
CREATE OR REFRESH MATERIALIZED VIEW crowd2fund.silver.withdraw_funds(
  CONSTRAINT valid_Amount EXPECT (Amount IS NOT NULL) ON VIOLATION DROP ROW
)
AS
SELECT Date,
       Time,
       -1 * Amount_Out_GBP as Amount
FROM crowd2fund.bronze.transactions
WHERE From_To = 'Withdrawal Request'

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW crowd2fund.silver.add_withdraw_requests
AS
SELECT * FROM crowd2fund.silver.add_funds
UNION ALL
SELECT * FROM crowd2fund.silver.withdraw_funds